In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import json
import requests
import time
import editdistance
import random

datadir = pathlib.Path("/home/david/pro/scc/data")

%run utils.py

# Geografisk afstand
Det burde være nemt! Jeg gør det lige nemmere at pakke!

In [ ]:
with open("tmp.json", "r", encoding="utf-8") as fd:
    known, dump, notfound, nonunique = json.load(fd)

In [ ]:
with (datadir / "gps-coords.csv").open("w", encoding="utf-8") as fd:
    fd.write("location|lat|lon\n")
    for place, data in sorted(known.items()):
        fd.write(place + "|" + str(data["lat"]) + "|" + str(data["lng"]) + "\n")

In [ ]:
with (datadir / "gps-coords.csv").open("r", encoding="utf-8") as fd:
    next(fd)
    _gps = {}
    for line in fd:
        place, lat, lon = line.split("|")
        _gps[place] = (float(lat), float(lon))

In [ ]:
import math

def geo_dist(lat1, lon1, lat2, lon2):
    # pasta
    R = 6371e3 # Jordens radius i meter
    φ1 = math.radians(lat1)
    φ2 = math.radians(lat2)
    Δφ = math.radians(lat2-lat1)
    Δλ = math.radians(lon2-lon1)

    a = math.sin(Δφ/2) * math.sin(Δφ/2) + \
            math.cos(φ1) * math.cos(φ2) * \
            math.sin(Δλ/2) * math.sin(Δλ/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c;

In [ ]:
def geo_similarity(loc1, loc2, max_dist=6e5):
    if not (loc1 in _gps and loc2 in _gps):
        return 0
    dist = geo_dist(*_gps[loc1], *_gps[loc2])
    return max(0, 1 - dist/max_dist)

In [ ]:
geo_similarity("københavn", "kjöbenhavn")

In [ ]:
geo_similarity("københavn", "svenborg")

In [ ]:
geo_similarity("københavn", "sverige")

In [ ]:
geo_similarity("odense", "svenborg")

# String similarity

Den her er ikke så nice

In [ ]:
def dum_version(a,b):
    d = editdistance.eval(a,b)
    return 1 / (1 + (d/(len(a) + len(b))))

Den her virker mere fornuftig

In [ ]:
def anden_dum_version(a, b):
    d = editdistance.eval(a, b)
    return 1 - d / (max(len(a), len(b)))

In [ ]:
anden_dum_version("københavn", "kiøbenhavn")

In [ ]:
anden_dum_version("københavn", "københavnstrup")

In [ ]:
dum_version("kjøbenhavn", "københavn")

In [ ]:
dum_version("vejen", "vojens")

In [ ]:
civilstande = collections.Counter()
for fn in sorted(datadir.glob("lc_*.csv")):
    year = int(re.search(r"\d{4}", fn.name).group(0))
    print(year)
    with fn.open("r", encoding="UTF-8") as fd:
        print(next(fd))
        #continue
        for line in fd:
            row = line.strip().split("|")
            civilstande.update((row[10],))

In [ ]:
civilstande.most_common(100)

# Fødselsår

In [ ]:
def year_similarity(y1, y2, max_dist=4):
    # lineært skalering med max
    return max(0, 1 - (abs(y1 - y2)/max_dist))

In [ ]:
year_similarity(1819, 1822)

# Samlet?

In [ ]:
def similarity(row1, row2):
    